In [1]:
import numpy as np

class EventList:
    def __init__(self):
        self.time = []
        self.entity = []
        self.event = []
        self.prevtime = 0
    
    def __str__(self):
        res = str(self.prevtime)+ "\n"
        res += "# : Time  \t     Customer \t\t       Event " +"\n"
        for i in range(len(self.time)):
            res+=f"{i:<2.0f}"+":"+" {:<8.3f}\t {:<8} \t{:>10}".format(float(self.time[i]),str(self.entity[i]),self.event[i])+"\n"
        return res
    
    def __repr__(self):
        res = ""
        for i in range(len(self.time)):
            res+=f"{i:<2.0f}"+":"+" {:<8.3f}\t {:<8} \t{:>10}".format(float(self.time[i]),str(self.entity[i]),self.event[i])+"\n"
        return res
    
    def addEvent(self,time,entity,event):
        """
        Adds an event to the EventList object. The data stored is the time the event occurred, the entity that did something?, and the event that happened (eg time 0, server, served a customer)
        """
        curr = time
        self.time.append(time)
        self.time.sort()
        idx = self.time.index(curr)
        self.entity.insert(idx,entity)
        self.event.insert(idx,event)
        
    def next(self):
        """
        Modifies the event list, such that the first event is deleted and then returns the event after it. (eg if event list is [(0, server, served customer A), (2.344, server, served customer B)], the first element is deleted and the second element is returned)
        """
        time = self.time[0]
        entity = self.entity[0]
        event = self.event[0]
        self.prevtime = time
        del (self.time[0])
        del (self.entity[0])
        del (self.event[0])
        return time,entity,event

    def __len__(self):
        return len(self.event)

In [2]:
class Customer:
    '''
    Customer object, only attribute is arrival time
    '''
    def __init__(self, arriv_time, number):
        '''
        arriv_time - when customer arrives at store
        number - used to identify customer
        '''
        self.Atime = arriv_time
        self.number = number
        self.Dtime = -1
        
    
    def __repr__(self):
        return f"Customer({self.time:.3f})"
    
    def __str__(self):
        return f"{self.number}:{self.Atime:.3f},{self.Dtime:.3f}"
    
    def getTime(self):
        '''
        Returns arrival time
        '''
        return self.Atime
    
    def __eq__(self,other):
        if type(other) != Customer:
            return False
        return self.number == other.number


In [3]:
class Server:
    def __init__(self, server_number, service_rate):
        '''
        server_number - used as a name to identify server
        service_rate - how fast server operates
        '''
        self.busy = False
        self.number = server_number
        self.rate = service_rate
        self.customer = None
        self.customer_number = None
            
    
    def __repr__(self):
        return f"Server {self.number}: Customer=>{self.customer}; Busy=>{self.busy}; ServiceRate=>{1/self.rate:.3f}"
            
    
    def isBusy(self):
        '''
        Boolean of whether or not server is busy
        '''
        return self.busy
        
    def freeUp(self):
        '''
        Makes server not busy
        '''
        self.busy = False
        self.customer = None
        self.customer_number = None
        
    def makeBusy(self):
        '''
        Changes server to busy
        '''
        self.busy = True
        
    def serve(self, cust, arrive_time):
        '''
        Used to schedule a customer's departure
        Inputs:
            cust - customer
            arrive_time - time customer gets to server
            serve_rate - how fast server serves
        Outputs:
            A scheduled time based on arrival time, server rate, and the
            exponential distribution
        '''
        self.makeBusy()
        self.customer = cust
        self.customer_number = cust.number
        np.random.seed(220)
        time_to_serve = np.random.exponential(self.rate,1)[0]
        cust.Dtime = cust.Atime + time_to_serve
        return cust.Dtime

    def __lt__(self,other):
        return self.rate < other.rate
    
    def __gt__(self,other):
        return self.rate > other.rate

In [4]:
def create_customer_Arrival(arrival_rate, max_time):
    '''
    Creates an EventList object of customers' scheduled arrivals
    Inputs:
        arrival_rate - how fast customers arrive (using the exponential distribution)
        max_time - for how long customers arrive
    Outputs:
        EventList of customers' scheduled arrivals
    '''
    np.random.seed(220)
    t = 0
    cust = 1
    arriv = 0
    customerlist = EventList()
    customerlist.addEvent(arriv,Customer(arriv,cust),"Arrival")
    while arriv <= max_time:
        cust+=1
        rand_exp = np.random.exponential(arrival_rate,1)[0]
        arriv = rand_exp+t
        t+= rand_exp
        customerlist.addEvent(arriv,Customer(arriv,cust),"Arrival")
    
    return customerlist

In [5]:
def is_available(server_lst):
    """
    Takes in a list of servers and outputs a bool determining whether a server is available and/or the index of that server in the list
    input: server_lst- a list of server objects
    output:  [False ] or [True, index of server]
    
    """

    for i, server in enumerate(server_lst):
        if not server.isBusy():
            return [True, i]
    else:       
        return [False]
    

In [6]:
def find_customer_server(server_lst,customer):
    '''
    Takes in a list of server objects and a customer object
    input: server_lst - list of server objects
           customer - customer object
    output: Server object that contains the server
    
    '''
    

    for server in server_lst:
        if customer.number == server.customer_number:
            return server
        

In [7]:
def simulateShop(filename, num_servers, server_rates):
    '''
    Writes a simulation of a queue to specified outfile
    given a specified number of servers and corresponding
    server rates.
    Inputs:
        filename - string that you want to output file to be called
        num_servers - number of servers in the "shop"
        server_rates - service rates for the servers
    Outputs:
        No output, writes simulation to file
    '''
    outfile =  open(filename+".txt","w+")
    Queue = EventList()
    customer_event = create_customer_Arrival(5,100)
    
    server_lst = []
    for x in range(num_servers):
        server_lst.append(Server(x+1,server_rates[x]))
    while len(customer_event):

        event_time, customer, event = customer_event.next()

        if event == "Arrival":
            available = is_available(sorted(server_lst))

            if available[0]:
                server = server_lst[available[1]]
                end_time = server.serve(customer,event_time)
                customer_event.addEvent(end_time,customer,"Departure")
            elif not available[0]:
                Queue.addEvent(event_time , customer, event)
        elif event == "Departure":
            server = find_customer_server(server_lst,customer)
            server.freeUp()
            if len(Queue):
                event_time , customer, event = Queue.next()
                available = is_available(sorted(server_lst))
                server = server_lst[available[1]]
                end_time = server.serve(customer,event_time)
                customer_event.addEvent(end_time,customer,"Departure")
        print(customer_event, file= outfile)
        for x in server_lst:
            print(x, file = outfile)
        print("=====Customer Line ===============\n", file = outfile)
        if event_time >=100:
            break



    outfile.close()

In [8]:
filename = input("Outfile name?")
num_servers = int(input("How many servers do you want?"))
server_rates = [0 for i in range(num_servers)]
for i in range(num_servers):
    server_rates[i] = int(input(f"What rate do you want server {i+1} to work at?"))
print("Queue simulation created as a file!")
simulateShop(filename, num_servers, server_rates)

Outfile name? q

How many servers do you want? 3

What rate do you want server 1 to work at? 10

What rate do you want server 2 to work at? 40

What rate do you want server 3 to work at? 50

Queue simulation created as a file!
